## Global Residualization

[Open in Colab](https://colab.research.google.com/github/mikarubi/abct/blob/main/docs-code/examples/1_residualn.ipynb){.btn .btn-dark .btn-sm}

Many datasets and networks contain dominant global patterns that sometimes represent artifactual, trivial, or irrelevant structure. Correspondingly, analyses often seek to remove these patterns to uncover or accentuate interesting underlying structure. We use the term _global residualization_ to describe this removal.

Here, we show approximate equivalence between three variants of global residualization across unsupervised learning, network science, and imaging neuroscience:

- _First-component removal_: Subtraction of rank-one approximation of the data (common in unsupervised learning).
- _Degree correction_: Subtraction of the normalized outer product of node degree vectors (common in network science).
- _Global signal regression_: Regression of the mean time series signal from the time series data (common in imaging neuroscience).

### Set up and load data

In [ ]:
# Install abct and download abct_utils.py
base = "https://github.com/mikarubi/abct/raw/refs/heads/main"
!wget --no-clobber {base}/docs-code/examples/abct_utils.py
%pip install --quiet abct nilearn

# Import modules
import abct
import numpy as np
from abct_utils import W, X, C, ordw, ordc, not_eye, fig_scatter, fig_imshow

### Show original structural and correlation networks

To show these relationships, we first consider the original structural and correlation networks in our example brain-imaging data. Each network contains 360 nodes (rows and columns) that denote cortical (brain) regions.

In [ ]:
fig_imshow(W[np.ix_(ordw, ordw)],
           "Original structural network",
           "inferno").show()

fig_imshow(C[np.ix_(ordc, ordc)],
           "Original correlation network",
           "viridis").show()

### Residualize structural and correlation networks

We now apply the three variants of global residualization to these networks. Note that while the degree and first component are removed directly from the networks, the global signal is regressed out of the time series data.

In [ ]:
# Residualize structural network
Wd = abct.residualn(W, "degree")
Wr = abct.residualn(W, "rankone")

# Residualize correlation networks
Cd = abct.residualn(C, "degree")
Xg = abct.residualn(X.T, "global").T
Xg = Xg / np.linalg.norm(Xg, axis=0, keepdims=True)
Cg = Xg.T @ Xg

### Visualize residual structural and correlation networks

We now visualize variants of the residual structural and correlation networks.

In [ ]:
fig_imshow(Wd[np.ix_(ordw, ordw)],
           "Struct. degree correction",
           "inferno").show()

fig_imshow(Wr[np.ix_(ordw, ordw)],
           "Struct. first-component removal",
           "inferno").show()

fig_imshow(Cd[np.ix_(ordc, ordc)],
           "Corr. degree correction",
           "viridis").show()

fig_imshow(Cg[np.ix_(ordc, ordc)],
           "Corr. global signal regression",
           "viridis").show()

Finally, we directly show the strong similarity between network weights after distinct residualizations.

In [ ]:
r = np.corrcoef(Wr[not_eye], Wd[not_eye])[0, 1]
fig_scatter(Wr[not_eye], Wd[not_eye], 
            "Weights after first-component removal",
            "Weights after degree correction",
            f"Structural network (r = {r:.3f})").show()

r = np.corrcoef(Cg[not_eye], Cd[not_eye])[0, 1]
fig_scatter(Cg[not_eye], Cd[not_eye], 
            "Weights after global-signal regression",
            "Weights after degree correction",
            f"Correlation network (r = {r:.3f})").show()